## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-14-01-16-00 +0000,wsj,Cautious RBA in No Rush to Cut Rates Further,https://www.wsj.com/articles/cautious-rba-in-n...
1,2025-10-14-01-10-51 +0000,nyt,Live Updates: Hostages and Palestinian Prisone...,https://www.nytimes.com/live/2025/10/12/world/...
2,2025-10-14-01-06-31 +0000,nyt,"Trump Takes a Victory Lap, but Avoids Question...",https://www.nytimes.com/2025/10/13/world/middl...
3,2025-10-14-01-04-00 +0000,wsj,Singapore’s Central Bank Extends Monetary Poli...,https://www.wsj.com/economy/central-banking/si...
4,2025-10-14-01-00-00 +0000,wsj,How China and the U.S. Are Racing to De-Escala...,https://www.wsj.com/world/china/trump-tariffs-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2340/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,54
9,hostages,22
31,china,21
384,gaza,20
150,israel,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
94,2025-10-13-20-53-37 +0000,wapo,Trump and world leaders meet in Egypt on futur...,https://www.washingtonpost.com/world/2025/10/1...,141
91,2025-10-13-20-57-45 +0000,bbc,Israeli hostages and Palestinian detainees fre...,https://www.bbc.com/news/articles/c740jx07vz0o...,131
102,2025-10-13-20-45-44 +0000,nypost,Dems largely quiet as Trump headlines historic...,https://nypost.com/2025/10/13/us-news/top-dems...,128
147,2025-10-13-17-58-10 +0000,nypost,Trump boasts Gaza peace process could be presi...,https://nypost.com/2025/10/13/us-news/trump-sa...,124
134,2025-10-13-18-40-17 +0000,nypost,Trump trolls Norway rep over Nobel Peace Prize...,https://nypost.com/2025/10/13/us-news/trump-tr...,124


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
94,141,2025-10-13-20-53-37 +0000,wapo,Trump and world leaders meet in Egypt on futur...,https://www.washingtonpost.com/world/2025/10/1...
253,68,2025-10-13-10-31-45 +0000,nypost,"Israeli twin brothers Ziv, Gali Berman reunite...",https://nypost.com/2025/10/13/world-news/israe...
218,56,2025-10-13-13-11-27 +0000,nypost,"Dow soars nearly 600 points after Trump, Vance...",https://nypost.com/2025/10/13/business/dow-fut...
62,53,2025-10-13-21-59-00 +0000,wsj,What has Trump done as president so far? The W...,https://www.wsj.com/politics/policy/donald-tru...
129,49,2025-10-13-19-03-00 +0000,wsj,"Joel Mokyr, Philippe Aghion and Peter Howitt w...",https://www.wsj.com/economy/nobel-economics-pr...
101,45,2025-10-13-20-47-44 +0000,nypost,NYC bodega clerk locked up for infamous 1979 E...,https://nypost.com/2025/10/13/us-news/nyc-bode...
118,45,2025-10-13-19-54-27 +0000,bbc,'Peace in the Middle East': Trump signs ceasef...,https://www.bbc.com/news/videos/c8r0xm84e4go?a...
104,36,2025-10-13-20-39-04 +0000,nypost,NJ Gov. Murphy’s camp silent as critics rip hi...,https://nypost.com/2025/10/13/us-news/nj-gov-m...
184,36,2025-10-13-16-00-00 +0000,wsj,The relationship between capitalism and democr...,https://www.wsj.com/politics/united-states-ame...
60,35,2025-10-13-22-02-37 +0000,latimes,"Fire survivors to get up to $350,000 for perso...",https://www.latimes.com/business/story/2025-10...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
